In [1]:
import requests
import json
import multiprocessing as mp
import mag_functions as F

In [3]:
input_json_multi = \
"""[
    {
        "title": "         Applicability of Magnetic resonance imaging of the knee in FORENSIC age estimation       ",
        "doc_type": "Journal",
        "journal": "american journal of forensic medicine and pathology"
    },
    {
        "title": "counterfactual reward modification for streaming recommendation with delayed feedback",
        "doc_type": "Conference",
        "journal": null
    },
    {
        "title": "modification of silicone elastomer with zwitterionic silane for durable antifouling properties",
        "doc_type": null,
        "journal": null
    }
]"""

# Define your api URL here 
# headers = {"X-API-Key": api_key}
# res = requests.post(request_url, json=input_json_multi, headers=headers) 
# output_api = res.json()
# for i in output_api:
#     print(i['tags'])

### Load Testing

In [3]:
input_json_single = \
"""[
   {
       "title": "applicability of magnetic resonance imaging of the knee in forensic age estimation",
       "doc_type": "Journal",
       "journal": "american journal of forensic medicine and pathology"
       
   }
]"""


In [6]:
%%time
F.get_tags(input_json_multi, 1)[1]

CPU times: user 22.2 ms, sys: 3.86 ms, total: 26.1 ms
Wall time: 460 ms


[{'tags': ['medicine',
   'magnetic resonance imaging',
   'nuclear medicine',
   'forensic anthropology',
   'radiology']},
 {'tags': ['computer science',
   'counterfactual thinking',
   'task',
   'reinforcement learning',
   'artificial intelligence']},
 {'tags': ['materials science',
   'silicone',
   'silicone rubber',
   'silane',
   'elastomer',
   'siloxane',
   'composite material',
   'biofouling',
   'chemical engineering',
   'coating',
   'polydimethylsiloxane',
   'surface modification',
   'contact angle',
   'silane coupling',
   'filler',
   'silanes',
   'polymer']}]

In [13]:
test_payload = [input_json_single]*8000

In [87]:
%%time
final_output = []
with mp.Pool(processes=8) as p:
    results = [p.apply(F.get_tags, args=(x, i)) for i,x in enumerate(test_payload)]

CPU times: user 5.34 s, sys: 1.4 s, total: 6.74 s
Wall time: 45min 40s


In [93]:
%%time
final_output = []
with mp.Pool(processes=8) as p:
    results = [p.apply_async(F.get_tags, args=(x, i)) for i,x in enumerate(test_payload)]
    output = [r.get(timeout=300) for r in results]

final_output += output

# print("Done with first")
    
# with mp.Pool(processes=8) as p:
#     results = [p.apply_async(F.get_tags, args=(x, i)) for i,x in enumerate(test_payload)]
#     output = [r.get(timeout=300) for r in results]

# final_output += output

TimeoutError: 

In [89]:
# output.sort()

In [90]:
len(final_output)

0

### Testing the enhanced model

In [1]:
import os
import io
import boto3
import json

In [2]:
ENDPOINT_NAME = 'mag-imitator-endpoint'
runtime= boto3.client('runtime.sagemaker')

In [4]:
response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='application/json',
                                       Body=input_json_multi)

In [5]:
print(response)

{'ResponseMetadata': {'RequestId': '0f9721f9-1a9e-45bd-920f-cbb665926547', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0f9721f9-1a9e-45bd-920f-cbb665926547', 'x-amzn-invoked-production-variant': 'variant-name-1', 'date': 'Sat, 01 Jan 2022 15:25:56 GMT', 'content-type': 'application/json', 'content-length': '1435'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'variant-name-1', 'Body': <botocore.response.StreamingBody object at 0x1186f64f0>}


In [6]:
result = json.loads(response['Body'].read().decode())

In [10]:
print(result)

[{'tags': ['medicine', 'magnetic resonance imaging', 'nuclear medicine', 'forensic anthropology', 'radiology'], 'scores': [0.5264689922332764, 0.4692302346229553, 0.39589232206344604, 0.3734455108642578, 0.3555443286895752], 'tag_ids': [71924100, 143409427, 2989005, 176979668, 126838900]}, {'tags': ['computer science', 'counterfactual thinking', 'task', 'reinforcement learning', 'artificial intelligence'], 'scores': [0.6955634951591492, 0.4845692217350006, 0.39626574516296387, 0.38736552000045776, 0.3649846613407135], 'tag_ids': [41008148, 108650721, 2780451532, 97541855, 154945302]}, {'tags': ['materials science', 'silicone', 'silicone rubber', 'silane', 'elastomer', 'siloxane', 'composite material', 'biofouling', 'chemical engineering', 'coating', 'polydimethylsiloxane', 'surface modification', 'contact angle', 'silane coupling', 'filler', 'silanes', 'polymer'], 'scores': [0.6924267411231995, 0.6599656343460083, 0.584079384803772, 0.5673587322235107, 0.5134502649307251, 0.46404856443